In [169]:
import pandas as pd
import numpy as np
from scipy.stats.distributions import gamma

from epimodel import read_csv
from modeling.classes.class_populations import ascertainment


In [170]:
data = read_csv('../../../epimodel-covid-data/johns-hopkins.csv')
data = data.loc[data.index.dropna(),:]
cities = pd.read_excel('/home/guest/Downloads/Coronavirus Brazil.xlsx',sheet_name='Cities')
regions = 
with open('../../../epifor/data/regions.yaml','r') as infile:
    regions_raw = yaml.load(infile)
regions = pd.io.json.json_normalize(regions_raw,['subregions','subregions','subregions'])
regions = regions.rename(columns={'key':'GV Region'})

/home/guest/.cache/pypoetry/virtualenvs/epimodel-98U75rOO-py3.7/lib64/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """
/home/guest/.cache/pypoetry/virtualenvs/epimodel-98U75rOO-py3.7/lib64/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [171]:
data_low = ascertainment(data,fatality=0.004)
data_mid = ascertainment(data,fatality=0.008)
data_high = ascertainment(data,fatality=0.012)

In [172]:
asc_low, asc_mid, asc_high = (data_low[data_low['New deaths']>=5].loc['BR','Ascertainment'].mean(),
                             data_mid[data_high['New deaths']>=5].loc['BR','Ascertainment'].mean(),
                             data_high[data_high['New deaths']>=5].loc['BR','Ascertainment'].mean())

In [173]:
cities = pd.merge(cities,regions[['GV Region','population','gleam_id']],on='GV Region')

cities['population'] = cities['population'].fillna(cities['Population']*(cities['population']/cities['Population']).mean())
cities['Proportional cases (official)'] = np.maximum(cities['Proportional cases (official)'],1)

In [174]:
cities['GV proportional'] = cities['Proportional cases (official)'].astype(float)*cities['population']/cities['Population']

In [237]:
cities[cities['Recovered Low']/cities['population']>0.2]

,City,Population,State,Share of state,Proportional cases (official),Proportional cases (estimated),Proportional deaths,GV Region,population,gleam_id,GV proportional,Infectious Low,Infectious Mid,Infectious High,Exposed Low,Exposed Mid,Exposed High,Recovered Low,Recovered Mid,Recovered High
3,Araçatuba,1100000.0,São Paulo,0.025057,234.808656,36953.085950,17.414579,aracatuba,170000.0,2037.0,36.288610,7275.057122,3637.528561,2425.019041,23897.883805,11948.941902,7965.961268,46835.517923,23417.758962,15611.839308
5,Araraquara,1200000.0,São Paulo,0.027335,256.154898,40312.457400,18.997722,araraquara,168000.0,2048.0,35.861686,7936.425953,3968.212976,2645.475318,26070.418701,13035.209350,8690.139567,51093.292289,25546.646144,17031.097430
14,Caçador,953537.0,Santa Catarina,0.159917,132.091052,20787.870790,4.157830,cacador,64000.0,1940.0,8.865757,4092.566116,2046.283058,1364.188705,13443.697811,6721.848906,4481.232604,26347.209441,13173.604720,8782.403147
24,Coari,132724.0,Amazonas,0.035836,53.180236,8369.256293,3.225216,coari,53000.0,2047.0,21.236193,1647.678835,823.839418,549.226278,5412.471228,2706.235614,1804.157076,10607.461952,5303.730976,3535.820651
32,Francisco Beltrão,825149.0,Paraná,0.076655,60.633903,9542.279493,2.759571,francisco beltrao,58000.0,1988.0,4.261977,1878.615185,939.307593,626.205062,6171.075588,3085.537794,2057.025196,12094.188911,6047.094456,4031.396304
37,Itacoatiara,260656.0,Amazonas,0.070378,104.440399,16436.340590,6.333986,itacoatiara,52000.0,2022.0,20.835510,3235.868227,1617.934113,1078.622742,10629.525183,5314.762592,3543.175061,20831.941493,10415.970747,6943.980498
40,Joaçaba,324432.0,Santa Catarina,0.054410,44.942739,7072.877608,1.414662,joacaba,39000.0,2036.0,5.402571,1392.457147,696.228573,464.152382,4574.091796,2287.045898,1524.697265,8964.390322,4482.195161,2988.130107
43,Juazeiro Do Norte,3100000.0,Ceará,0.340659,683.021978,107490.798200,36.450549,juazeiro do norte,225000.0,2008.0,49.574176,21162.013318,10581.006659,7054.004439,69515.239108,34757.619554,23171.746369,112500.000000,68118.630372,45412.420248
53,Marília,1200000.0,São Paulo,0.027335,256.154898,40312.457400,18.997722,marilia,212000.0,1964.0,45.254032,7936.425953,3968.212976,2645.475318,26070.418701,13035.209350,8690.139567,51093.292289,25546.646144,17031.097430
57,Navegantes,1600000.0,Santa Catarina,0.268334,221.643925,34881.282280,6.976685,navegantes,48000.0,2033.0,6.649318,6867.175356,3433.587678,2289.058452,22558.030263,11279.015132,7519.343421,24000.000000,22104.823995,14736.549330


In [252]:
new_to_infectious = (data_low.loc['BR','Confirmed']/asc_low/data_low.loc['BR','Infectious'])[-30:].mean()

In [177]:
new_to_exposed = (data_low.loc['BR','New cases true'].shift(1)/data_low.loc['BR','Exposed'])[-30:].mean()

In [178]:
new_to_recovered = (data_low.loc['BR','New cases true'].shift(5)/data_low.loc['BR','Recovered'])[-30:].mean()

ratios = {'Infectious':new_to_infectious, 'Exposed':new_to_exposed, 'Recovered':new_to_recovered, 
          'Low':asc_low, 'Mid': asc_mid, 'High': asc_high}

In [263]:
def annotateCities(quant,bound):
    cities[f'{quant} {bound}'] = cities['Proportional cases (official)'].diff()/ratios[bound]/ratios[quant]

for quant in ['Infectious','Exposed','Recovered']:
    for bound in ['Low','Mid','High']:
        annotateCities(quant,bound)


In [257]:
cities[pd.isna(cities["Infectious Mid"])]

,City,Population,State,Share of state,Proportional cases (official),Proportional cases (estimated),Proportional deaths,GV Region,population,gleam_id,GV proportional,Infectious Low,Infectious Mid,Infectious High,Exposed Low,Exposed Mid,Exposed High,Recovered Low,Recovered Mid,Recovered High
0,Alta Floresta,254025.0,Mato Grosso do Sul,0.120671,1.0,0.0,0.0,alta floresta,40000.0,1942.0,0.157465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
out_inf = '\n'.join([f'<seed city="{int(c)}" compartment="Infectious" number="{int(n)}" />' for c,n in cities[['gleam_id','Infectious Mid']].values])
out_exp = '\n'.join([f'<seed city="{int(c)}" compartment="Exposed" number="{int(n)}" />' for c,n in cities[['gleam_id','Exposed Mid']].values])
out_rec = '\n'.join([f'<seed city="{int(c)}" compartment="Recovered" number="{int(n)}" />' for c,n in cities[['gleam_id','Recovered Mid']].values])


ValueError: cannot convert float NaN to integer

In [259]:
print('\n'.join([out_inf,out_exp,out_rec]))

<seed city="1942" compartment="Infectious" number="30" />
<seed city="1970" compartment="Infectious" number="302" />
<seed city="1977" compartment="Infectious" number="1394" />
<seed city="2037" compartment="Infectious" number="7275" />
<seed city="1962" compartment="Infectious" number="348" />
<seed city="2048" compartment="Infectious" number="7936" />
<seed city="2023" compartment="Infectious" number="626" />
<seed city="1956" compartment="Infectious" number="1125" />
<seed city="1999" compartment="Infectious" number="9920" />
<seed city="1937" compartment="Infectious" number="6588" />
<seed city="2034" compartment="Infectious" number="10487" />
<seed city="2039" compartment="Infectious" number="3501" />
<seed city="1961" compartment="Infectious" number="30" />
<seed city="1939" compartment="Infectious" number="4107" />
<seed city="1940" compartment="Infectious" number="4092" />
<seed city="1950" compartment="Infectious" number="399" />
<seed city="1973" compartment="Infectious" numb

In [229]:
out_inf = '\n'.join([f'<seed city="{int(c)}" compartment="Infectious" number="{int(n)}" />' for c,n in cities[['gleam_id','Infectious High']].values])
out_exp = '\n'.join([f'<seed city="{int(c)}" compartment="Exposed" number="{int(n)}" />' for c,n in cities[['gleam_id','Exposed High']].values])
out_rec = '\n'.join([f'<seed city="{int(c)}" compartment="Recovered" number="{int(n)}" />' for c,n in cities[['gleam_id','Recovered High']].values])
print('\n'.join([out_inf,out_exp,out_rec]))

<seed city="1942" compartment="Infectious" number="10" />
<seed city="1970" compartment="Infectious" number="100" />
<seed city="1977" compartment="Infectious" number="464" />
<seed city="2037" compartment="Infectious" number="2425" />
<seed city="1962" compartment="Infectious" number="116" />
<seed city="2048" compartment="Infectious" number="2645" />
<seed city="2023" compartment="Infectious" number="208" />
<seed city="1956" compartment="Infectious" number="375" />
<seed city="1999" compartment="Infectious" number="3306" />
<seed city="1937" compartment="Infectious" number="2196" />
<seed city="2034" compartment="Infectious" number="3495" />
<seed city="2039" compartment="Infectious" number="1167" />
<seed city="1961" compartment="Infectious" number="10" />
<seed city="1939" compartment="Infectious" number="1369" />
<seed city="1940" compartment="Infectious" number="1364" />
<seed city="1950" compartment="Infectious" number="133" />
<seed city="1973" compartment="Infectious" number=

In [230]:
out_inf = '\n'.join([f'<seed city="{int(c)}" compartment="Infectious" number="{int(n)}" />' for c,n in cities[['gleam_id','Infectious Low']].values])
out_exp = '\n'.join([f'<seed city="{int(c)}" compartment="Exposed" number="{int(n)}" />' for c,n in cities[['gleam_id','Exposed Low']].values])
out_rec = '\n'.join([f'<seed city="{int(c)}" compartment="Recovered" number="{int(n)}" />' for c,n in cities[['gleam_id','Recovered Low']].values])
print('\n'.join([out_inf,out_exp,out_rec]))

<seed city="1942" compartment="Infectious" number="30" />
<seed city="1970" compartment="Infectious" number="302" />
<seed city="1977" compartment="Infectious" number="1394" />
<seed city="2037" compartment="Infectious" number="7275" />
<seed city="1962" compartment="Infectious" number="348" />
<seed city="2048" compartment="Infectious" number="7936" />
<seed city="2023" compartment="Infectious" number="626" />
<seed city="1956" compartment="Infectious" number="1125" />
<seed city="1999" compartment="Infectious" number="9920" />
<seed city="1937" compartment="Infectious" number="6588" />
<seed city="2034" compartment="Infectious" number="10487" />
<seed city="2039" compartment="Infectious" number="3501" />
<seed city="1961" compartment="Infectious" number="30" />
<seed city="1939" compartment="Infectious" number="4107" />
<seed city="1940" compartment="Infectious" number="4092" />
<seed city="1950" compartment="Infectious" number="399" />
<seed city="1973" compartment="Infectious" numb

In [264]:
cities['Infectious Mid'].sum()

1490.3242247330782

In [261]:
cities['Infectious Mid'].sum()

1490.3242247330782

# Effectiveness estimates from

https://imperialcollegelondon.github.io/covid19estimates/#/interventions

Assumption is that original $R_0$ is around 4.

If we assume $R_0 = 2.6$ and final value after all measures are implemented is about 0.9 as ICL find, we get 25% less effectiveness (estimate)

Our $R_0$ might be a bit low, not updating this for now

EF estimates suggests school closures are substantially more effective and lockdown substantially less. Not sure where this difference is coming from, but rating school a bit better here and lockdown/public events a bit worse

In [207]:
# cancel public events, self isolation, school closure, full lockdown

reds_ICL = np.array([0.15,.05,.1,.7])

# cancel public events, self isolation, school closure, business cancellation, full lockdown


reds = np.array([0.15,0.05,.25,.25,.5])

In [217]:
reds_GLEAM = reds*.75
BETA = 1.285

In [218]:
# Mild: cancel public events, self isolation

(1-reds_GLEAM[:2]).prod()*BETA

1.09767109375

In [219]:
# Moderate: cancel public events, self isolation, school closure, business cancellation

(1-reds_GLEAM[:4]).prod()*BETA

0.7246344329833984

In [220]:
# Full: cancel public events, self isolation, school closure, business cancellation, full lockdown

(1-reds_GLEAM).prod()*BETA

0.45289652061462393

In [247]:
cities[['Infectious Low','Recovered Low','Exposed Low']].sum()

Infectious Low    7.170172e+05
Recovered Low     4.563510e+06
Exposed Low       2.355335e+06
dtype: float64

In [248]:
asc_low

0.03347000113489193

In [253]:
new_to_infectious

3.053160680498956

In [250]:
cities[['Infectious Mid','Recovered Mid','Exposed Mid']].sum()

Infectious Mid    3.585086e+05
Recovered Mid     2.308014e+06
Exposed Mid       1.177667e+06
dtype: float64